In [17]:
import pandas as pd
from os.path import join
from feature.cat_dataset import make_dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')
holiday_path = join('jeju_data', '국가공휴일.csv')

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

x_train, y_train, test = make_dataset(train_path, test_path, holiday_path)

Start time:  2022-10-31 17:57:17.063403
Train dataset success !
Test dataset success !
End time: 2022-10-31 17:58:35.945202
Play time:  0:01:18.881799


In [18]:
X = x_train.copy()
y = y_train.copy()

In [30]:
X_array = X.values
y_array = y.values

In [35]:
params = {'learning_rate': 0.04814728699783611, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3736, 'max_depth': 16, 'random_strength': 59, 'colsample_bylevel': 0.4861875826015146, 'l2_leaf_reg': 2.309535224342252e-05, 'min_child_samples': 58, 'max_bin': 424, 'od_type': 'Iter'}
cat_model = CatBoostRegressor(**params).fit(X, y)
y_pred = cat_model.predict(test)

sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_cat_t.csv", index=False)

0:	learn: 15.5032311	total: 449ms	remaining: 27m 56s
1:	learn: 15.2673161	total: 598ms	remaining: 18m 36s
2:	learn: 14.8164798	total: 1.91s	remaining: 39m 30s
3:	learn: 14.3778236	total: 3.34s	remaining: 51m 55s
4:	learn: 13.9726213	total: 4.79s	remaining: 59m 38s
5:	learn: 13.6174878	total: 6.07s	remaining: 1h 2m 50s
6:	learn: 13.2508268	total: 7.47s	remaining: 1h 6m 21s
7:	learn: 12.9339620	total: 8.78s	remaining: 1h 8m 10s
8:	learn: 12.6294124	total: 10.2s	remaining: 1h 10m 42s
9:	learn: 12.3216023	total: 11.7s	remaining: 1h 12m 29s
10:	learn: 12.0352395	total: 13s	remaining: 1h 13m 16s
11:	learn: 11.7632932	total: 14.4s	remaining: 1h 14m 31s
12:	learn: 11.6083813	total: 14.6s	remaining: 1h 9m 46s
13:	learn: 11.3777823	total: 15.9s	remaining: 1h 10m 32s
14:	learn: 11.1419635	total: 17.2s	remaining: 1h 11m 9s
15:	learn: 10.9524381	total: 17.6s	remaining: 1h 8m 14s
16:	learn: 10.7969872	total: 17.9s	remaining: 1h 5m 11s
17:	learn: 10.6811684	total: 18.1s	remaining: 1h 2m 17s
18:	learn

In [36]:
params = {'learning_rate': 0.04814728699783611, 'bagging_temperature': 40.52044595178952, 'n_estimators': 3736, 'max_depth': 16, 'random_strength': 59, 'colsample_bylevel': 0.4861875826015146, 'l2_leaf_reg': 2.309535224342252e-05, 'min_child_samples': 58, 'max_bin': 424, 'od_type': 'Iter'}


from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

CAT_model= {}

for f in range(5):
      print(f'===================================={f+1}============================================')
      train_idx, val_idx = folds[f]
      
      x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
      
      CAT = CatBoostRegressor(**params, cat_features=cat_features)
      CAT.fit(x_train, y_train)
      
      y_pred = CAT.predict(x_val)
      mae = mean_absolute_error(y_val, y_pred)
      print(f"{f + 1} Fold MAE = {mae}")
      CAT_model[f] = CAT
      print(f'================================================================================\n\n')
              

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
for fold in range(5):
    sample_submission['target'] += CAT_model[fold].predict(test)/5    
sample_submission.to_csv("./catcat.csv", index=False)

c:\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


====================================1============================================
0:	learn: 15.4775128	total: 1.48s	remaining: 1h 32m 12s
1:	learn: 15.0758656	total: 4.35s	remaining: 2h 15m 23s
2:	learn: 14.6612380	total: 7.33s	remaining: 2h 32m 2s
3:	learn: 14.2700726	total: 8.71s	remaining: 2h 15m 28s
4:	learn: 13.9105438	total: 11.7s	remaining: 2h 25m 38s
5:	learn: 13.6150299	total: 13s	remaining: 2h 14m 16s
6:	learn: 13.3098638	total: 15.7s	remaining: 2h 19m 33s
7:	learn: 12.9868290	total: 17.6s	remaining: 2h 16m 20s
8:	learn: 12.7076483	total: 18.6s	remaining: 2h 8m 12s
9:	learn: 12.4297922	total: 21.4s	remaining: 2h 12m 52s
10:	learn: 12.2962540	total: 21.9s	remaining: 2h 3m 32s
11:	learn: 12.0733559	total: 23s	remaining: 1h 58m 42s
12:	learn: 11.8329333	total: 24.3s	remaining: 1h 56m 6s
13:	learn: 11.6126112	total: 27.4s	remaining: 2h 1m 27s
14:	learn: 11.4380409	total: 28.4s	remaining: 1h 57m 34s
15:	learn: 11.3192502	total: 28.9s	remaining: 1h 52m 4s
16:	learn: 11.2182167	tota